In [4]:
%matplotlib inline

#
# import utilitaires
#
import matplotlib.pyplot as plt

import helpers as hlp
import helpers.features as feat
import helpers.dataset.PlantVillage as pv
import helpers.split as split
import helpers.visual_words as vw

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [5]:
#
# parametres
#
configs = hlp.get_configs("config_overrides.json")


#
# dataset PlantVillage
#
pv_dataset = pv.load(configs.plant_village)
pv_dataframe = pv_dataset.dataframe

print("Dataset", pv_dataframe.shape)
print()


#
# train/test
#
train, test = split.load(configs.split)

print("Train size:", len(train))
print("Test size:", len(test))
print()


#
# orb
#
orb_bovw = vw.load_bovw(configs.orb_bovw, None)
orb_pv_vw = vw.load_dataset_vw(configs.orb_dataset_vw, \
                               None, \
                               None, \
                               None)
    
print("ORB BoVW clusters:", orb_bovw.model.cluster_centers_.shape)
print("ORB PlantVillage Visual Words:", orb_pv_vw.h5_vw_freqs.shape)
print()


#
# sift
#
sift_bovw = vw.load_bovw(configs.sift_bovw, None)
sift_pv_vw = vw.load_dataset_vw(configs.sift_dataset_vw, \
                                None, \
                                None, \
                                None)

print("Sift BoVW clusters:", sift_bovw.model.cluster_centers_.shape)
print("Sift PlantVillage Visual Words:", sift_pv_vw.h5_vw_freqs.shape)

Dataset (60343, 5)

Train size: 3711
Test size: 928

ORB BoVW clusters: (500, 32)
ORB PlantVillage Visual Words: (3711, 500)

Sift BoVW clusters: (500, 128)
Sift PlantVillage Visual Words: (3711, 500)


In [16]:
#
# mettre le data dans une forme acceptable pour sklear
#
y = pv_dataframe.loc[train, "label"]

words = sift_pv_vw.h5_index_to_vw_freqs.keys()

# display( sift_pv_vw.h5_file.keys() )

In [ ]:
#
# entrainer KNN
#
# on veut cross validation pour determiner le meilleur K et la meilleur accuracy
# knn_classifier = KNeighborsClassifier(n_neighbors=5)